In [1]:
import pandas as pd
import numpy as np
# if using Anaconda, install scikit-surprise to your conda environment using:
# conda install -c conda-forge scikit-surprise (pip install sckit-surprise yields errors...)
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import dump


# define current working folder
# curr_dir = os.path.dirname(os.path.realpath(__file__))
curr_dir = os.getcwd()

In [2]:
# user = pd.read_csv('BX-Users.csv', sep=';', error_bad_lines=False, encoding="latin-1")
# user.columns = ['userID', 'Location', 'Age']
# rating = pd.read_csv('BX-Book-Ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")
# rating.columns = ['userID', 'ISBN', 'bookRating']

# import tables from Instacart Kaggle dataset
aisles = pd.read_csv(os.path.join(curr_dir, 'aisles.csv'))
departments = pd.read_csv(os.path.join(curr_dir, 'departments.csv'))
order_products_prior = pd.read_csv(os.path.join(curr_dir, 'order_products__prior.csv'))
order_products_train = pd.read_csv(os.path.join(curr_dir, 'order_products__train.csv'))
orders = pd.read_csv(os.path.join(curr_dir, 'orders.csv'))
products = pd.read_csv(os.path.join(curr_dir, 'products.csv'))

# stack the previous train/"prior" datasets used for training/test
# we want to redo the train/test groups and not rely on the Kaggle 
# dataset creator's groups since we're going to drop data
order_products = order_products_train
order_products = order_products.append(order_products_prior)

# print('\nORDER_PRODUCT_DUPS')
# print(order_products_dups)

del order_products_prior
del order_products_train

# merge the tables together because there are no duplicates on primary keys 
all_merged_data = pd.merge(order_products, products, how='left', on=['product_id', 'product_id'])
all_merged_data = pd.merge(all_merged_data, orders, how='left', on=['order_id', 'order_id'])
all_merged_data = pd.merge(all_merged_data, aisles, how='left', on=['aisle_id', 'aisle_id'])
all_merged_data = pd.merge(all_merged_data, departments, how='left', on=['department_id', 'department_id'])

del aisles
del departments
del orders
del products


In [3]:
# CUT DOWN ON PRODUCTS
# Need to reduce the number of products in the user/products matrix for recommendations (123k users x 49k products = ~110GB of RAM required for float distance calculations)

# get the products and the number of orders they were sold in
product_dist = all_merged_data.groupby(['product_id','product_name']).size().sort_values(ascending=False).reset_index()
product_dist.columns = ['product_id','product_name','count']

# loop through the count of products by order they were sold in to see
# information about products with very low sales (like % of volume) 
for i in range(1,21):
    threshold = 40 * i

    pct_of_prod_ids_under_threshold = 100 * product_dist['product_id'].loc[product_dist['count'] <= threshold].count() / product_dist['product_id'].count()
    
    pct_of_prod_volume_sold_under_threshold = 100 * product_dist['count'].loc[product_dist['count'] <= threshold].sum() / product_dist['count'].sum()

    print(str(round(pct_of_prod_ids_under_threshold,2)) + '% of product codes have ' + str(threshold) + \
            ' or fewer sales accounting for ' + str(round(pct_of_prod_volume_sold_under_threshold,2)) + '% of all sales')


41.28% of product codes have 40 or fewer sales accounting for 1.01% of all sales
54.97% of product codes have 80 or fewer sales accounting for 2.18% of all sales
62.22% of product codes have 120 or fewer sales accounting for 3.23% of all sales
67.15% of product codes have 160 or fewer sales accounting for 4.24% of all sales
70.53% of product codes have 200 or fewer sales accounting for 5.14% of all sales
73.19% of product codes have 240 or fewer sales accounting for 6.0% of all sales
75.42% of product codes have 280 or fewer sales accounting for 6.85% of all sales
77.33% of product codes have 320 or fewer sales accounting for 7.69% of all sales
78.92% of product codes have 360 or fewer sales accounting for 8.48% of all sales
80.2% of product codes have 400 or fewer sales accounting for 9.19% of all sales
81.35% of product codes have 440 or fewer sales accounting for 9.91% of all sales
82.32% of product codes have 480 or fewer sales accounting for 10.56% of all sales
83.24% of product c

In [4]:
# CUT DOWN ON USERS
# Need to reduce the number of users in the user/products matrix for recommendations (123k users x 49k products = ~110GB of RAM required for float distance calculations)

# get number of unique orders by user
user_order_dist = all_merged_data.groupby('user_id')['order_id'].nunique().reset_index()
user_order_dist.columns = ['user_id','order_count']
user_order_dist = user_order_dist.sort_values(by='order_count',ascending=False)
user_order_dist

# get number of product rows (products are counted for each order they appear in) - this is the number of rows removed from the data
user_order_prod_count_dist = all_merged_data.groupby('user_id')['product_id'].count().reset_index()
user_order_prod_count_dist.columns = ['user_id','product_by_order_count']
user_order_prod_count_dist = user_order_prod_count_dist.sort_values(by='product_by_order_count',ascending=False)
user_order_prod_count_dist

# merge the order count with the total rows
user_dist = pd.merge(user_order_dist,user_order_prod_count_dist, how='inner', on='user_id').reset_index()

# loop through order counts by user and show product sold (count them for each order)
# to see where to cut the data off
for i in range(1,20):

    threshold = i

    pct_of_user_ids_under_threshold = 100 * user_dist['user_id'].loc[user_dist['order_count'] <= threshold].count() / user_dist['user_id'].count()
    pct_of_user_volume_sold_under_threshold = 100 * user_dist['order_count'].loc[user_dist['order_count'] <= threshold].sum() / user_dist['order_count'].sum()

    pct_of_sales_volume_sold_under_threshold = 100 * user_dist['product_by_order_count'].loc[user_dist['order_count'] <= threshold].sum() / user_order_prod_count_dist['product_by_order_count'].sum()

    print(str(round(pct_of_user_ids_under_threshold,2)) + '% of users have ' + str(threshold) + \
            ' or fewer orders, or ' + str(round(pct_of_user_volume_sold_under_threshold,2)) + \
            '% of all orders' + ' or ' + str(round(pct_of_sales_volume_sold_under_threshold,2)) +'% of all product volume.')

del user_order_prod_count_dist

0.0% of users have 1 or fewer orders, or 0.0% of all orders or 0.0% of all product volume.
0.0% of users have 2 or fewer orders, or 0.0% of all orders or 0.0% of all product volume.
4.21% of users have 3 or fewer orders, or 0.78% of all orders or 0.74% of all product volume.
15.1% of users have 4 or fewer orders, or 3.46% of all orders or 3.29% of all product volume.
23.96% of users have 5 or fewer orders, or 6.19% of all orders or 5.92% of all product volume.
31.39% of users have 6 or fewer orders, or 8.94% of all orders or 8.59% of all product volume.
37.79% of users have 7 or fewer orders, or 11.7% of all orders or 11.28% of all product volume.
43.16% of users have 8 or fewer orders, or 14.35% of all orders or 13.88% of all product volume.
47.9% of users have 9 or fewer orders, or 16.98% of all orders or 16.48% of all product volume.
52.1% of users have 10 or fewer orders, or 19.56% of all orders or 19.05% of all product volume.
55.69% of users have 11 or fewer orders, or 22.0% of a

In [5]:
print('\n\nTOP PRODUCTS (BY ORDERS CONTAINING PRODUCT)')
print(product_dist.head(10))

print('\n\nTOP USERS (BY ORDERS CONTAINING PRODUCT)')
print(user_dist.head(10))




TOP PRODUCTS (BY ORDERS CONTAINING PRODUCT)
   product_id            product_name   count
0       24852                  Banana  491291
1       13176  Bag of Organic Bananas  394930
2       21137    Organic Strawberries  275577
3       21903    Organic Baby Spinach  251705
4       47209    Organic Hass Avocado  220877
5       47766         Organic Avocado  184224
6       47626             Large Lemon  160792
7       16797            Strawberries  149445
8       26209                   Limes  146660
9       27845      Organic Whole Milk  142813


TOP USERS (BY ORDERS CONTAINING PRODUCT)
   index  user_id  order_count  product_by_order_count
0      0   112841          100                     763
1      1   174555          100                     874
2      2    73676          100                     661
3      3   176469          100                     622
4      4    90584          100                     351
5      5   190487          100                     795
6      6    57367   

In [25]:
# DROP LOW VALUE RECORDS TO REDUCE 'DIMENSIONALITY'
# AND CONVERT THE REORDERING TO A 5-POINT SCALE SYSTEM BASED ON NUMBER OF REORDERS BY A USER 

print('Original dataframe shape:\t\t\t{}'.format(all_merged_data.shape))

# choose product sales volume threshold for inclusion in model and drop low-sale-count 
# products from all_merged data (i.e. not enough purchases to make good recommendations)
product_threshold = 400
product_dist.drop(product_dist['product_id'].loc[product_dist['count'] <= product_threshold].index, inplace=True)
all_merged_data_reduced = pd.merge(all_merged_data, product_dist['product_id'], how='inner', on='product_id')

# choose product sales volume threshold for inclusion in model and drop low-sale-count 
# users from all_merged data (likely not enough purchases to make good recommendations)
user_threshold = 15
user_dist.drop(user_dist['user_id'].loc[user_dist['order_count'] <= user_threshold].index, inplace=True)
all_merged_data_reduced = pd.merge(all_merged_data_reduced, user_dist['user_id'], how='inner', on='user_id')

# drop columns not necessary to recommendation engine
all_merged_data_reduced.drop(['order_id','add_to_cart_order','aisle_id','department_id','eval_set','order_number','order_dow','order_hour_of_day','days_since_prior_order','aisle','department'], axis=1, inplace=True)

# show the raw reduction in shape
print('Deleted users and products dataframe shape:\t {}'.format(all_merged_data_reduced.shape))

# group the resulting dataset on user_id and product_id 
# map the number of reorders by user/product to a 5-point scale 
model_data = all_merged_data_reduced.reset_index()

model_data = model_data.groupby(['user_id','product_id','product_name'])['reordered'].sum().reset_index()
model_data.columns = ['user_id','product_id','product_name','reordered_count']

def get_rating(row):
    if row['reordered_count'] == 0:
        return 1
    elif row['reordered_count'] == 1:
        return  2
    elif row['reordered_count'] <= 4:
        return 3
    elif row['reordered_count'] <= 9:
        return 4
    elif row['reordered_count'] <= 99:
        return 5
    else:
        return 0

model_data['rating'] = model_data.apply (lambda row: get_rating(row), axis=1)
model_data['rating_obj'] = model_data['rating'].astype(object) # for pivot table category

model_data = model_data.drop(['reordered_count'], axis=1)

print('Grouped model dataframe shape:\t\t\t {}'.format(model_data.shape))


Original dataframe shape:			(33819106, 15)
Deleted users and products dataframe shape:	 (21455288, 4)
Grouped model dataframe shape:			 (6709570, 5)


In [26]:
pivot = pd.pivot_table(model_data, index=['product_name'], columns=['rating_obj'], values=['rating'], aggfunc='count', fill_value=0)
pivot

rating                   
rating_obj                                        1    2    3    4   5
product_name                                                          
#2 Coffee Filters                               246   71   41    8   1
0% Fat Blueberry Greek Yogurt                    78   18   19   14   8
0% Fat Free Organic Milk                        183   73  118   66  69
0% Fat Organic Greek Vanilla Yogurt             150   48   52   38  26
0% Fat Superfruits Greek Yogurt                  93   26   28   13  10
...                                             ...  ...  ...  ...  ..
of Hanover 100 Calorie Pretzels Mini            301  164  160   64  40
smartwater® Electrolyte Enhanced Water          328  114  146   81  78
vitaminwater® XXX Acai Blueberry Pomegranate    223   65   74   34  23
with Crispy Almonds Cereal                      264  123  152  100  76
with Olive Oil Mayonnaise                       162   38   21    1   0

[9837 rows x 5 columns]

In [27]:
model_data.drop(['product_name','rating_obj'], axis=1, inplace=True)
model_data

,user_id,product_id,rating
0,7,274,1
1,7,519,2
2,7,4920,4
3,7,4945,3
4,7,6361,3
...,...,...,...
6709565,206208,48017,1
6709566,206208,48364,1
6709567,206208,48865,1
6709568,206208,49247,1


In [28]:
model_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6709570 entries, 0 to 6709569
Data columns (total 3 columns):
 #   Column      Dtype
---  ------      -----
 0   user_id     int64
 1   product_id  int64
 2   rating      int64
dtypes: int64(3)
memory usage: 153.6 MB


In [29]:
# clean up once you've sufficiently reduced the dimensionality above
del product_dist
del user_dist
del all_merged_data
del all_merged_data_reduced

In [104]:
print('Dataset shape: {}'.format(model_data.shape))
print('-Dataset examples-')
print(model_data.iloc[::500000, :])

Dataset shape: (6709570, 3)
-Dataset examples-
         user_id  product_id  scale
0              7         274      1
500000     15553       14992      4
1000000    30947       22935      5
1500000    45991       37436      4
2000000    61399       27336      5
2500000    76460         160      1
3000000    91631       41375      1
3500000   106863       14371      5
4000000   122166         890      4
4500000   137916       17122      5
5000000   153163        8048      1
5500000   168950        7901      5
6000000   184461       26709      3
6500000   199737       42768      3


In [98]:
# save the merged data as a csv (so that it can be imported later without needing to re-merge)
model_data.to_csv(os.path.join(curr_dir,'model_data.csv'), sep=',', encoding='utf-8', index=False)

# load the merged data as a csv (so that it can be imported later without needing to re-merge)
# model_data = pd.read_csv(os.path.join(curr_dir,'model_data.csv'))


In [6]:
# load the data into Surprise using Reader
reorder = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(model_data[['user_id', 'product_id', 'rating']], reorder)


In [29]:
# define the trainset and testset
trainset, testset = train_test_split(data, test_size=0.25)


In [56]:
# use the SVD algorithm
algo = SVD(n_epochs = 20, n_factors = 50, verbose = True)

# train the model on trainset and validate predictions for known ratings on the testset
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions)

Estimating biases using sgd...
RMSE: 1.1388


1.1388271656447095

In [57]:
# show the first n predictions for the trainset (actual and estimated rating)
predictions[0:10]


[Prediction(uid=133467, iid=21137, r_ui=1.0, est=1.9286465477468584, details={'was_impossible': False}),
 Prediction(uid=178888, iid=45007, r_ui=2.0, est=2.4434226423476555, details={'was_impossible': False}),
 Prediction(uid=79046, iid=44625, r_ui=5.0, est=3.0983893867160845, details={'was_impossible': False}),
 Prediction(uid=104042, iid=37570, r_ui=1.0, est=1.5305284267455972, details={'was_impossible': False}),
 Prediction(uid=176283, iid=16797, r_ui=3.0, est=2.4323762463254037, details={'was_impossible': False}),
 Prediction(uid=2476, iid=11073, r_ui=3.0, est=2.715419083321947, details={'was_impossible': False}),
 Prediction(uid=193275, iid=39928, r_ui=3.0, est=2.039766100330349, details={'was_impossible': False}),
 Prediction(uid=131792, iid=20632, r_ui=5.0, est=2.405726807603433, details={'was_impossible': False}),
 Prediction(uid=203160, iid=4472, r_ui=1.0, est=2.395816870653601, details={'was_impossible': False}),
 Prediction(uid=49307, iid=34448, r_ui=2.0, est=2.7535834444953

In [3]:
# define the trainset to be the entire dataset 
# (to generate predictions for ALL the products a user has not purchased)
trainset = data.build_full_trainset()

# define the model
algo = SVD(n_epochs = 20, n_factors = 50, verbose = True)

# train the model
algo.fit(trainset)


Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


In [97]:
# loop through all users and all product combinations in model_data and add recommendations to a dataframe
# PRODUCTS WITH SUFFICIENT SALES WILL BE RECOMMENDED TO USERS WITH SUFFICIENT HISTORY

# all distinct users
all_users = model_data['user_id'].unique()
all_users = pd.DataFrame(all_users, columns=['user_id']).reset_index()

# all distinct products
model_products = model_data['product_id'].unique()
model_products = pd.DataFrame(model_products, columns=['product_id'])
model_products = pd.merge(model_products, products, how='inner', on='product_id')

recommendations = pd.DataFrame(columns=['user_id','product_id','product_name','model_rating'])

for u in all_users.itertuples():
    if u[0] % 1000 == 0:
        print('records processed:\t' + str(u[0]))

    # for selected user, rate all products they haven't bought before
    user_ratings = []
    for i in model_products.itertuples():
        dict = {'user_id':u[2],
                    'product_id':i[1],
                    'product_name':i[2],
                    'aisle_id':i[3],
                    'department_id':i[4],
                    'model_rating': algo.predict(u[2], i[1], verbose=False)[3] 
                }
        user_ratings.append(dict)

    df = pd.DataFrame(user_ratings).sort_values(by='model_rating',ascending=False).reset_index()
    df = df.groupby('aisle_id').first().sort_values(by='model_rating',ascending=False).reset_index()
    df = df.drop(['aisle_id','index','department_id'],axis=1)
    df = df.iloc[0:10,]
    recommendations = recommendations.append(df)

recommendations.to_csv(os.path.join(curr_dir,'recommendations.csv'))


7
records processed:	0
15
17
21
24
27
28
29
31
36
37
42
46
50
52
54
63
67
71
75
82
86
87
89
90
91
96
99
101
103
110
112
118
122
127
132
133
138
140
142
146
150
152
153
154
155
161
168
173
174
175
182
187
190
195
197
204
206
208
209
210
214
216
219
220
221
222
223
226
227
229
232
233
234
235
236
242
243
248
256
257
258
260
262
264
270
271
273
278
280
281
284
289
290
294
295
300
301
304
306


In [84]:
recommendations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 0 to 9
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   user_id       200 non-null    object 
 1   product_id    200 non-null    object 
 2   product_name  200 non-null    object 
 3   model_rating  200 non-null    float64
dtypes: float64(1), object(3)
memory usage: 7.8+ KB
